# Compare GP solvers

In [8]:
import numpy as np
import matplotlib.pyplot as pl

np.random.seed(1234)
x = np.sort(np.random.uniform(0, 10, 50000))
yerr = 0.1 * np.ones_like(x)
y = np.sin(x)

n = 10000

### george GP

In [9]:
import george
from george import kernels
kernel = np.var(y) * kernels.ExpSquaredKernel(1.0)

gp_basic = george.GP(kernel)
%timeit gp_basic.compute(x[:n], yerr[:n])
print(gp_basic.lnlikelihood(y[:n]))

3.57 s ± 23.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
13811.700746056806


### george fast GP solver

In [5]:
n_big = n
gp_hodlr = george.GP(kernel, solver=george.HODLRSolver)
%timeit gp_hodlr.compute(x[:n_big], yerr[:n_big])
print(gp_hodlr.lnlikelihood(y[:n_big]))


3.6 ms ± 241 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1374.571335372935


### scikit-learn equivalent (without hyperparameter tuning)

In [7]:
import sklearn
print("sklearn version: {0}".format(sklearn.__version__))
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process import GaussianProcessRegressor

kernel_skl = np.var(y) * RBF(length_scale=1.0)
gp_skl = GaussianProcessRegressor(kernel_skl,
                                  alpha=yerr[:n]**2,
                                  optimizer=None,
                                  # optimizer='fmin_l_bfgs_b',
                                  copy_X_train=False)
%timeit gp_skl.fit(x[:n, None], y[:n])
print(gp_skl.log_marginal_likelihood(kernel_skl.theta))

sklearn version: 1.2.2
37.6 ms ± 1.01 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
1374.5708082218775
